<a href="https://colab.research.google.com/github/desaiankitb/pytorch-basics/blob/main/tutorial-pytorch-org/01_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quickstart

- This section runs through the API for commons in machine learning. Refer to the links in each section to dive deeper. 

## Working with data 

- PyTorch has two `premitives to work with` [data](<https://pytorch.org/docs/stable/data.html>):

- `torch.utils.data.DataLoader` and `torch.utils.data.Dataset`. `Dataset` stores the samples and their corresponding lables, and `DataLoader` wraps an iterable around the `Dataset`. 



In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets 
from torchvision.transforms import ToTensor, Lambda, Compose 
import matplotlib.pyplot as plt 

- PyTorch offers domain-specific libraries such as [TorchText](<https://pytorch.org/text/stable/index.html>), [TorchVision](https://pytorch.org/vision/stable/index.html), and [TorchAudio](https://pytorch.org/audio/stable/index.html) all of which includes datasets. For this tutorial, we will be using a TorchVision dataset. 

- The `torchvision.dataset` module contains `Dataset` objects for many real-world vision data like CIFAR, COCO ([full-list](https://pytorch.org/vision/stable/datasets.html)). In this turorial, we use the FashionMNIST dataset. Every TorchVision `Dataset` includes two arguments: `transform` and `target_transform` to modify the samples and lables respectively. 


In [2]:
# Download training data from open datasets. 
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets. 
test_data = datasets.FashionMNIST(
    root = "data",
    train=False,
    download=True,
    transform=ToTensor(),
)


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


- We pass `Dataset` as an argument to `DataLoader`. This wraps an iterable over dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading. Here, we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels. 

In [3]:
batch_size = 64

# Create data loaders. 
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print("Shape of X [N, C, H, W]", X.shape)
  print("Shape of y:", y.shape, y.dtype)
  break

Shape of X [N, C, H, W] torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Creating Models
- To define a neural network in PyTorch, we create a class that inherits from [`nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). We define the layers of the network in the `__init__` function and specify how data will pass through the network in the `forward` function. To accelerate operations in the neural network, we move it to GPU if available. 


In [6]:
# Get cpy or gpu device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.ReLU()
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


# Optimising the Model Parameters

- To train a model, we need a [`loss function`](https://pytorch.org/docs/stable/nn.html#loss-functions) and an [`optimizer`](https://pytorch.org/docs/stable/optim.html). 

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

- In a single training loop, the model makes predictions on the training dataset (fed to in batches), and backpropogates the prediction error to adjust the model's parameters. 

In [8]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y), in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error 
    pred = model(X)
    loss = loss_fn(pred, y)
    
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

- We also check the model's performence against the test dataset to ensure it is learning.

In [9]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size 
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

- The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions. We print the model's accuracy and loss at each epoch;  we would like to see the accuracy increase and the loss decrease with every epoch. 

In [10]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1} \n ----------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1 
 ----------------------------
loss: 2.302682 [    0/60000]
loss: 2.301892 [ 6400/60000]
loss: 2.293572 [12800/60000]
loss: 2.285275 [19200/60000]
loss: 2.290929 [25600/60000]
loss: 2.267130 [32000/60000]
loss: 2.270980 [38400/60000]
loss: 2.266323 [44800/60000]
loss: 2.260579 [51200/60000]
loss: 2.216217 [57600/60000]
Test Error: 
 Accuracy: 22.6%, Avg loss: 2.243885 

Epoch 2 
 ----------------------------
loss: 2.254046 [    0/60000]
loss: 2.258748 [ 6400/60000]
loss: 2.230551 [12800/60000]
loss: 2.219502 [19200/60000]
loss: 2.227423 [25600/60000]
loss: 2.178536 [32000/60000]
loss: 2.189924 [38400/60000]
loss: 2.174086 [44800/60000]
loss: 2.192852 [51200/60000]
loss: 2.079284 [57600/60000]
Test Error: 
 Accuracy: 37.4%, Avg loss: 2.140635 

Epoch 3 
 ----------------------------
loss: 2.175437 [    0/60000]
loss: 2.169520 [ 6400/60000]
loss: 2.113851 [12800/60000]
loss: 2.106301 [19200/60000]
loss: 2.105894 [25600/60000]
loss: 2.026064 [32000/60000]
loss: 2.052443 [38400/600